# Harmonizome ETL: Mouse Gene Ontology (MGI)

Created by: Charles Dai <br>
Credit to: Moshe Silverstein

Data Source: http://www.informatics.jax.org/downloads/reports/index.html

In [ ]:
# appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [ ]:
import sys
import os
from datetime import date

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import harmonizome.utility_functions as uf
import harmonizome.lookup as lookup

In [ ]:
%load_ext autoreload
%autoreload 2

### Notebook Information

In [ ]:
print('This notebook was run on:', date.today(), '\nPython version:', sys.version)

# Initialization

In [ ]:
%%appyter hide_code

{% do SectionField(
    name='data',
    title='Upload Data',
    img='load_icon.png'
) %}

In [ ]:
%%appyter code_eval

{% do DescriptionField(
    name='description',
    text='The examples below were sourced from <a href="http://www.informatics.jax.org/downloads/reports/index.html" target="_blank">www.informatics.jax.org</a>. If clicking on the examples does not work, they should be downloaded directly from the source website.',
    section='data'
) %}

{% set df_file = FileField(
    constraint='.*\.rpt$',
    name='gene_phenotype', 
    label='Genotypes and Phenotype Annotations (rpt)', 
    default='Input/MGI/MGI_GenePheno.rpt',
    examples={
        'MGI_GenePheno.rpt': 'http://www.informatics.jax.org/downloads/reports/MGI_GenePheno.rpt'
    },
    section='data'
) %}

{% set gene_file = FileField(
    constraint='.*\.rpt$',
    name='gene_metadata', 
    label='MGI Genetic Markers (rpt)', 
    default='Input/MGI/MRK_GXDAssay.rpt',
    examples={
        'MRK_GXDAssay.rpt': 'http://www.informatics.jax.org/downloads/reports/MRK_GXDAssay.rpt'
    },
    section='data'
) %}

{% set ontology_file = FileField(
    constraint='.*\.rpt$',
    name='phenotype_vocabulary', 
    label='Mammalian Phenotype Vocabulary (rpt)', 
    default='Input/MGI/VOC_MammalianPhenotype.rpt',
    examples={
        'VOC_MammalianPhenotype.rpt': 'http://www.informatics.jax.org/downloads/reports/VOC_MammalianPhenotype.rpt'
    },
    section='data'
) %}

### Load Mapping Dictionaries

In [ ]:
symbol_lookup, geneid_lookup = lookup.get_lookups()

### Output Path

In [ ]:
output_name = 'mgi'

path = 'Output/MGI'
if not os.path.exists(path):
    os.makedirs(path)

# Load Data

In [ ]:
%%appyter code_exec

df = pd.read_csv(
    {{df_file}}, 
    header=None, usecols=[4, 6], names=['Phenotype ID', 'MGI Marker'], sep='\t')

In [ ]:
df.head()

In [ ]:
df.shape

## Load Gene Metadata

In [ ]:
%%appyter code_exec

gene_meta = pd.read_csv(
    {{gene_file}}, 
    header=None, usecols=[0, 1], index_col=0, sep='\t')

In [ ]:
gene_meta.head()

In [ ]:
gene_meta.shape

# Load Ontology Metadata

In [ ]:
%%appyter code_exec

ontology = pd.read_csv(
    {{ontology_file}}, 
    header=None, usecols=[0, 1], index_col=0, sep='\t')

In [ ]:
ontology.head()

In [ ]:
ontology.shape

# Pre-process Data

## Split Marker Lists

In [ ]:
df = df.dropna()
df['MGI Marker'] = df['MGI Marker'].str.split(pat='\|')
df = df.explode('MGI Marker')
df.head()

In [ ]:
df.shape

## Map Marker to Gene

In [ ]:
df['MGI Marker'] = gene_meta.reindex(df['MGI Marker'])[1].reset_index(drop=True)
df.head()

## Map Phenotype ID to Phenotype

In [ ]:
df['Phenotype ID'] = ontology.reindex(df['Phenotype ID'])[1].reset_index(drop=True)
df = df.set_index('MGI Marker')
df.index.name = 'Gene Symbol'
df.columns = ['Phenotype']
df.head()

# Filter Data

## Map Gene Symbols to Up-to-date Approved Gene Symbols

In [ ]:
df = uf.map_symbols(df, symbol_lookup, remove_duplicates=True)
df.shape

# Analyze Data

## Create Binary Matrix

In [ ]:
binary_matrix = uf.binary_matrix(df)
binary_matrix.head()

In [ ]:
binary_matrix.shape

In [ ]:
uf.save_data(binary_matrix, path, output_name + '_binary_matrix', 
            compression='npz', dtype=np.uint8)

## Create Gene List

In [ ]:
gene_list = uf.gene_list(binary_matrix, geneid_lookup)
gene_list.head()

In [ ]:
gene_list.shape

In [ ]:
uf.save_data(gene_list, path, output_name + '_gene_list',
            ext='tsv', compression='gzip', index=False)

## Create Attribute List

In [ ]:
attribute_list = uf.attribute_list(binary_matrix)
attribute_list.head()

In [ ]:
attribute_list.shape

In [ ]:
uf.save_data(attribute_list, path, output_name + '_attribute_list',
            ext='tsv', compression='gzip')

## Create Gene and Attribute Set Libraries

In [ ]:
uf.save_setlib(binary_matrix, 'gene', 'up', path, output_name + '_gene_up_set')

In [ ]:
uf.save_setlib(binary_matrix, 'attribute', 'up', path, 
                           output_name + '_attribute_up_set')

## Create Attribute Similarity Matrix

In [ ]:
attribute_similarity_matrix = uf.similarity_matrix(binary_matrix.T, 'jaccard', sparse=True)
attribute_similarity_matrix.head()

In [ ]:
uf.save_data(attribute_similarity_matrix, path,
            output_name + '_attribute_similarity_matrix', 
            compression='npz', symmetric=True, dtype=np.float32)

## Create Gene Similarity Matrix

In [ ]:
gene_similarity_matrix = uf.similarity_matrix(binary_matrix, 'jaccard', sparse=True)
gene_similarity_matrix.head()

In [ ]:
uf.save_data(gene_similarity_matrix, path, 
            output_name + '_gene_similarity_matrix',
            compression='npz', symmetric=True, dtype=np.float32)

## Create Gene-Attribute Edge List

In [ ]:
edge_list = uf.edge_list(binary_matrix)
uf.save_data(edge_list, path, output_name + '_edge_list', 
        ext='tsv', compression='gzip')

# Create Downloadable Save File

In [ ]:
uf.archive(path)

### Link to download output files: [click here](./output_archive.zip)